In [1]:
# use CEASELESS Kernel on adrien local laptop

import os
import xarray as xr
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import cartopy.crs as ccrs # => to add in Seastar env
import seastar as ss

from scipy import interpolate, stats # => to add in Seastar env ?

# from seastar.gmfs.doppler import *

In [2]:
aux = xr.Dataset(
        data_vars=dict(
            WindSpeed=(['GroundRange','CrossRange'], np.full([10,10], 5) ),
            WindDirection=(['GroundRange','CrossRange'], np.full([10,10], 90) ),
            IncidenceAngleImage=(['GroundRange','CrossRange'], np.full([10,10], 30) ),
            LookDirection=(['GroundRange','CrossRange','Antenna'],
                           np.stack( (np.full([10,10], 45),
                                      np.full([10,10], 90),
                                      np.full([10,10], 135)
                                      ), axis=-1
                                   )
                          ),
            Polarization=(['GroundRange','CrossRange'], np.full([10,10], 1) ),
        ),
        coords=dict(
            Antenna=['Fore','Mid','Aft']
        ),
        attrs=dict(
            CentralWavenumber=270,
            CentralFreq=13.5 * 10**9,
        )
)
aux.Polarization[2,2] = 2

In [3]:
aux

<xarray.Dataset>
Dimensions:              (Antenna: 3, CrossRange: 10, GroundRange: 10)
Coordinates:
  * Antenna              (Antenna) <U4 'Fore' 'Mid' 'Aft'
Dimensions without coordinates: CrossRange, GroundRange
Data variables:
    WindSpeed            (GroundRange, CrossRange) int64 5 5 5 5 5 ... 5 5 5 5 5
    WindDirection        (GroundRange, CrossRange) int64 90 90 90 ... 90 90 90
    IncidenceAngleImage  (GroundRange, CrossRange) int64 30 30 30 ... 30 30 30
    LookDirection        (GroundRange, CrossRange, Antenna) int64 45 90 ... 135
    Polarization         (GroundRange, CrossRange) int64 1 1 1 1 1 ... 1 1 1 1 1
Attributes:
    CentralWavenumber:  270
    CentralFreq:        13500000000.0

In [4]:
aux.sel(Antenna='Fore')

<xarray.Dataset>
Dimensions:              (CrossRange: 10, GroundRange: 10)
Coordinates:
    Antenna              <U4 'Fore'
Dimensions without coordinates: CrossRange, GroundRange
Data variables:
    WindSpeed            (GroundRange, CrossRange) int64 5 5 5 5 5 ... 5 5 5 5 5
    WindDirection        (GroundRange, CrossRange) int64 90 90 90 ... 90 90 90
    IncidenceAngleImage  (GroundRange, CrossRange) int64 30 30 30 ... 30 30 30
    LookDirection        (GroundRange, CrossRange) int64 45 45 45 ... 45 45 45
    Polarization         (GroundRange, CrossRange) int64 1 1 1 1 1 ... 1 1 1 1 1
Attributes:
    CentralWavenumber:  270
    CentralFreq:        13500000000.0

In [5]:
wasv_fore = ss.gmfs.doppler.compute_wasv(aux.sel(Antenna='Fore'), gmf='mouche12')
wasv_fore

<xarray.Dataset>
Dimensions:  (CrossRange: 10, GroundRange: 10)
Dimensions without coordinates: CrossRange, GroundRange
Data variables:
    WASV     (CrossRange, GroundRange) float64 -0.8688 -0.8688 ... -0.8688

In [6]:
wasv_mid = ss.gmfs.doppler.compute_wasv(aux.sel(Antenna='Mid'), gmf='mouche12')
wasv_mid

<xarray.Dataset>
Dimensions:  (CrossRange: 10, GroundRange: 10)
Dimensions without coordinates: CrossRange, GroundRange
Data variables:
    WASV     (CrossRange, GroundRange) float64 -1.151 -1.151 ... -1.151 -1.151

In [8]:
wasv_aft = ss.gmfs.doppler.compute_wasv(aux.sel(Antenna='Aft'), gmf='mouche12')
wasv_aft

<xarray.Dataset>
Dimensions:  (CrossRange: 10, GroundRange: 10)
Dimensions without coordinates: CrossRange, GroundRange
Data variables:
    WASV     (CrossRange, GroundRange) float64 -0.8688 -0.8688 ... -0.8688

In [9]:
ss.gmfs.doppler.compute_wasv(aux, gmf='yurovsky19')
# without subselection of the Antenna => raise ERROR

Exception: ds_env.LookDirection need to be a 2D field. 
Use e.g. ds_env.sel(Antenna="Fore") to reduce to a 2D field.

In [10]:
ss.gmfs.doppler.compute_wasv(aux.sel(Antenna='Fore'), gmf='yurovsky19', swh = 1)

<xarray.Dataset>
Dimensions:  (CrossRange: 10, GroundRange: 10)
Dimensions without coordinates: CrossRange, GroundRange
Data variables:
    WASV     (CrossRange, GroundRange) float64 -0.4926 -0.4926 ... -0.4926